# **Introduction:**

This file serves to design and test a custom implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS). This will be trained and evaluated against the ANN designed previously. 

This ANFIS takes the load history, the distance to the task, and the total distance travelled thus far and performs inference about the suitability of a given robot for a task at hand. 

**Date Created:** 13/01/2025

**Date Modified:** 14/01/2025

# **Import Packages:** 

This section imports all the necessary packages for the ANFIS implementation. 

In [143]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import product
# from tensorflow.keras import layers, Sequential
# from tensorflow.keras.optimizers import Adam

# **Layer Function & Class Definitions:**

Need to define the membership function to be used first:

In [144]:
# triangular membership function:
def triangular_mf(x, params):
    a, b, c = params

    if x <= a or x >= c:
        return 0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b < x < c:
        return (c - x) / (c - b)
    elif x == b:
        return 1
    elif x == c:
        return 1

Now need to define the ANFIS class:

In [145]:
class ANFIS:
    # object constructor:
    def __init__(self, num_inputs, num_mfs, num_rules, params = None):
        # need to instantiate the object:
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_rules
        self.memberships = {}

        # must first assign the antecedent parameters:
        # if custom:
        if params is not None:
            for j in range(self.num_inputs):
                for i in range(self.num_mfs):
                    self.memberships[f'membership_{j+1}_{i+1}'] = params[j, i, :]
            print('params set to other than None')
        # if not custom, randomly initialize them:
        else:
            for j in range(self.num_inputs):
                for i in range(self.num_mfs):
                    a = np.random.uniform(low = -1.0, high = 1.0)
                    b = np.random.uniform(low = -1.0, high = 1.0)
                    c = np.random.uniform(low = -1.0, high = 1.0)

                    params = np.array([a, b, c])
                    self.memberships[f'membership_{j+1}_{i+1}'] = params

        print('model created!')

    # this is a plotting function to verify that the membership functions are correct:

    def plot_membership_functions(self, max_values):
        x1_max, x2_max, x3_max = max_values

        input_ranges = {
            0: np.linspace(0, x1_max, 1000),
            1: np.linspace(0, x2_max, 1000), 
            2: np.linspace(0, x3_max, 1000),
        }

        label_values = {
            0: 'Low',
            1: 'Medium',
            2: 'High'
        }

        # plot mfs for each input:
        for input_index in range(self.num_inputs):
            x_values = input_ranges[input_index]
            plt.figure(figsize=(12, 8))

            # plot each mfs for the selected input:
            for i in range(self.num_mfs):
                params = self.memberships[f'membership_{input_index+1}_{i+1}']
                y_values = [triangular_mf(x, params) for x in x_values]
                plt.plot(x_values, y_values, label = f'{label_values[i]}')
        
                plt.title(f'Membership Functions for Input X{input_index + 1}')
                plt.xlabel('Input Value')
                plt.ylabel('Degree of Membership')
                plt.legend()
                plt.grid(True)
        plt.show()
    
    # this is the first layer within the anfis, the membership layer:

    def membership_layer(self, inputs):

        membership_values = {}

        # for every input j:
        for j in range(self.num_inputs):
            # for every membership function for that input:
            for i in range(self.num_mfs):
                params = self.memberships[f'membership_{j+1}_{i+1}']
                x = inputs[j]
                membership_values[f'membership_{j+1}_{i+1}'] = triangular_mf(x, params)
        
        return membership_values
    
    # this is the second layer within the anfis, the firing strength layer:

    def firing_strength_layer(self, membership_values):

        firing_strengths = {}
        rules = product(range(self.num_mfs), repeat = self.num_inputs)  # pre-generate rule combinations

        for rule_index, combination in enumerate(rules, start = 1):
            w_k = 1.0
            for input_index, mf_index in enumerate(combination):
                w_k *= membership_values[f'membership_{input_index + 1}_{mf_index + 1}']
            firing_strengths[f'rule_{rule_index}'] = w_k

        return firing_strengths
   
    # this is the third layer within the anfis, the normalization layer:

    def normalization_layer(self, firing_strengths):
        normalized_firing_strengths = {}
        total_strength = sum(firing_strengths.values())

        normalized_index = 1
        for value in firing_strengths.values():
            w_bar_k = value / total_strength 
            normalized_firing_strengths[f'normalized_value_{normalized_index}'] = w_bar_k
            normalized_index += 1

        return normalized_firing_strengths
    
    # this is the fourth layer within the anfis, the rule consequent layer:

    # def consequent_layer(self, normalized_firing_strengths):
        # 

        



Test the class:

In [146]:
params = np.array([
    [  # Parameters for input 1
        [0, 0, 6],
        [5/6, 5, 55/6],
        [4, 10, 10]
    ],
    [  # Parameters for input 2
        [0, 0 , 15],
        [25/12, 12.5, 275/12],
        [10, 25, 25]
    ],
    [  # Parameters for input 3
        [0, 0, 30],
        [25/6, 25, 275/6],
        [15, 50, 50]
    ]
])

inputs = np.array([2, 12, 21])
print(f'inputs are: {inputs}\n')
max_values = np.array([10, 25, 50])

test = ANFIS(num_inputs = 3, num_mfs = 3, num_rules = 2, params = params)
membership_values = test.membership_layer(inputs)
firing_strengths = test.firing_strength_layer(membership_values)
normalized_firing_strengths = test.normalization_layer(firing_strengths)

inputs are: [ 2 12 21]

params set to other than None
model created!


In [147]:
mfs = pd.DataFrame(membership_values, index = [1])
mfs.head()

,membership_1_1,membership_1_2,membership_1_3,membership_2_1,membership_2_2,membership_2_3,membership_3_1,membership_3_2,membership_3_3
1,0.666667,0.28,0,0.2,0.952,0.133333,0.3,0.808,0.171429


In [148]:
fs = pd.DataFrame(firing_strengths, index = [1]).transpose()
fs.head(27)

,1
rule_1,0.040000
rule_2,0.107733
rule_3,0.022857
rule_4,0.190400
rule_5,0.512811
rule_6,0.108800
rule_7,0.026667
rule_8,0.071822
rule_9,0.015238
rule_10,0.016800


In [149]:
nfs = pd.DataFrame(normalized_firing_strengths, index = [1]).transpose()
nfs.head(27)

,1
normalized_value_1,0.025694
normalized_value_2,0.069202
normalized_value_3,0.014682
normalized_value_4,0.122303
normalized_value_5,0.329403
normalized_value_6,0.069888
normalized_value_7,0.017129
normalized_value_8,0.046135
normalized_value_9,0.009788
normalized_value_10,0.010791
